# 準備

## artのインストール

In [20]:
!pip3 install adversarial-robustness-toolbox

## ライブラリのインポート

In [21]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Conv2D, Lambda, Flatten
from keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()
from keras import backend as K

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## CIFAR10のロード・前処理

In [22]:
# CIFAR10のロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# CIFAR10のラベル。
classes = [str(i) for i in range(10)]
num_classes = len(classes)

In [23]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 次元追加
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Classifier(k-WTAなし)の作成



## モデル定義

In [30]:
# モデルの定義。
inputs = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), padding='SAME', activation='relu')(inputs)
x = Conv2D(32, (3, 3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(64, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(64, (3,3), padding='SAME', activation='relu')(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model = Model(inputs, y)

# モデルのコンパイル。
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
dropout_23 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 14, 14, 64)        3692

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## 学習の実行

## モデルの精度評価

In [31]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=10,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
59904/60000 [============================>.] - ETA: 0s - loss: 1.0298 - accuracy: 0.6319

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


60000/60000 [==============================] - 16s 268us/sample - loss: 1.0286 - accuracy: 0.6324 - val_loss: 0.2879 - val_accuracy: 0.9259
Epoch 2/10
60000/60000 [==============================] - 15s 254us/sample - loss: 0.1840 - accuracy: 0.9442 - val_loss: 0.1376 - val_accuracy: 0.9697
Epoch 3/10
60000/60000 [==============================] - 15s 250us/sample - loss: 0.1244 - accuracy: 0.9627 - val_loss: 0.1198 - val_accuracy: 0.9729
Epoch 4/10
60000/60000 [==============================] - 15s 248us/sample - loss: 0.0916 - accuracy: 0.9720 - val_loss: 0.0849 - val_accuracy: 0.9815
Epoch 5/10
60000/60000 [==============================] - 15s 246us/sample - loss: 0.0761 - accuracy: 0.9759 - val_loss: 0.0699 - val_accuracy: 0.9836
Epoch 6/10
60000/60000 [==============================] - 15s 250us/sample - loss: 0.0600 - accuracy: 0.9811 - val_loss: 0.0748 - val_accuracy: 0.9832
Epoch 7/10
60000/60000 [==============================] - 15s 248us/sample - loss: 0.0550 - accuracy: 0.9

In [32]:
model.evaluate(X_test, y_test)

[0.04027113765515387, 0.9905]

# Classifier(k-WTAあり)の作成

## k-WTAの作成

In [33]:
class KWTA(tf.keras.layers.Layer):
  def __init__(self, k=None, **kwargs) :
    super().__init__(**kwargs)
    self.k = k

  def call(self, x):
    n = x[0].shape[0]
    if self.k == None:
            k = n * 1 // 10
    else:
            k = self.k
    topk = tf.math.top_k(x[0], k=k)   # 上位k個の値を抽出
    topk_min = K.min(topk.values)         # topkの最小値を抽出
    comp = tf.dtypes.cast(x >= topk_min, tf.float32)   # topkの最小値以上の値の部分を1, より小さい値の部分を0にしたTensorを生成
    return tf.math.multiply(x,comp)        # x * compより、topkの最小値以上の値のみそのまま、他は0になるようにする

class KWTA2D(tf.keras.layers.Layer):
    def __init__(self, k=None, **kwargs):
        super().__init__(**kwargs)
        self.k = k
        
    def call(self, x):
        n = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        if self.k == None:
            k = n * 1 // 10
        else:
            k = self.k
        x_flatten = tf.reshape(x[0],  [n])
        topk = tf.math.top_k(x_flatten, k=k)
        topk_min = K.min(topk.values) 
        comp = tf.dtypes.cast(x >= topk_min, tf.float32)
        return tf.math.multiply(x,comp)

## モデル定義

In [34]:
# モデルの定義。
inputs = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), padding='SAME')(inputs)
x = KWTA2D()(x)
x = Conv2D(32, (3, 3), padding='SAME')(x)
x = KWTA2D()(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(64, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = Conv2D(64, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = Dropout(0.25)(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = Conv2D(128, (3,3), padding='SAME')(x)
x = KWTA2D()(x)
x = GlobalAveragePooling2D()(x)

x = Dense(1024)(x)
x = KWTA()(x)
x = Dropout(0.25)(x)
y = Dense(10, activation='softmax')(x)

model_kwta = Model(inputs, y)

# モデルのコンパイル。
model_kwta.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_kwta.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
kwt_a2d (KWTA2D)             (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
kwt_a2d_1 (KWTA2D)           (None, 28, 28, 32)        0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 14, 14, 32)        0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## 学習の実行

In [35]:
# 学習の実行。
model_kwta.fit(X_train, y_train,
          batch_size=512,
          epochs=10,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 0s - loss: 2.2562 - accuracy: 0.1886

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


60000/60000 [==============================] - 19s 311us/sample - loss: 2.2562 - accuracy: 0.1886 - val_loss: 2.2283 - val_accuracy: 0.1664
Epoch 2/10
60000/60000 [==============================] - 17s 289us/sample - loss: 2.0811 - accuracy: 0.2451 - val_loss: 1.7615 - val_accuracy: 0.3580
Epoch 3/10
60000/60000 [==============================] - 17s 288us/sample - loss: 0.9498 - accuracy: 0.6645 - val_loss: 0.4896 - val_accuracy: 0.8518
Epoch 4/10
60000/60000 [==============================] - 17s 291us/sample - loss: 0.2888 - accuracy: 0.9145 - val_loss: 0.1789 - val_accuracy: 0.9524
Epoch 5/10
60000/60000 [==============================] - 18s 292us/sample - loss: 0.1393 - accuracy: 0.9594 - val_loss: 0.1090 - val_accuracy: 0.9727
Epoch 6/10
60000/60000 [==============================] - 17s 290us/sample - loss: 0.1038 - accuracy: 0.9688 - val_loss: 0.0832 - val_accuracy: 0.9816
Epoch 7/10
60000/60000 [==============================] - 17s 290us/sample - loss: 0.0784 - accuracy: 0.9

## モデルの精度評価

In [36]:
model_kwta.evaluate(X_test, y_test)

[0.03130911228219047, 0.991]

# 敵対的攻撃

## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [37]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

### k-WTAなし

In [46]:
# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
# FGSMインスタンスの作成。
attack = FastGradientMethod(estimator=classifier, eps=0.20, targeted=False)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_adv = attack.generate(x=X_test)

### k-WTAあり

In [47]:
classifier_kwta = KerasClassifier(model=model_kwta, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)
attack_kwta = FastGradientMethod(estimator=classifier_kwta, eps=0.20, targeted=False)
X_adv_kwta = attack_kwta.generate(X_test)

## 敵対的画像に対するモデルの精度評価

### k-WTAなし

In [48]:
model.evaluate(X_adv, y_test)

[2.4208896518707275, 0.2452]

### k-WTAあり

In [49]:
model_kwta.evaluate(X_adv_kwta, y_test)

[0.4960899344563484, 0.8426]